In [24]:
from Util import get_openai_api_key
OPEN_API_KEY=get_openai_api_key()

In [25]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY, 
    }
}

In [26]:
from autogen import AssistantAgent,UserProxyAgent

#agent1

user_input=UserProxyAgent(
    name="user_input",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
    code_execution_config={
        "use_docker":False
    }
)
email_writer=AssistantAgent(
    name="email_writer",
    max_consecutive_auto_reply=3,
    llm_config=llm_config,
    system_message="You are a marketing copywriter. Generate a promotional email using the given campaign details. Use persuasive and friendly language.",
)
translator_writer= AssistantAgent(
    name="translator_writer",
    max_consecutive_auto_reply=3,
    llm_config=llm_config,
    system_message="You are a language translator. Translate the given email content into Hindi, Spanish, and French while keeping the tone and formatting intact.",
)
reviewer_writer=AssistantAgent(
    name="reviewer_writer",
    max_consecutive_auto_reply=3,
    llm_config=llm_config,
    system_message="You are a professional proofreader. Correct any grammar mistakes, optimize the subject line, and enhance the call-to-action."
)
preview_sender=AssistantAgent(
    name="preview_sender",
    max_consecutive_auto_reply=3,
    llm_config=llm_config,
    system_message="You simulate sending a preview email to the manager. Output the preview content and ask: 'Do you approve this email for dispatch? (yes/no)'"
)
dispatcher = AssistantAgent(
    name="FinalDispatcherAgent",
    max_consecutive_auto_reply=3,
    llm_config=llm_config,
    system_message="If the manager approves, format and simulate sending the email to 10,000 customers. Confirm the dispatch and close the loop.",
)


In [27]:
user_campaign_input = """
Here are the campaign details:
- Product name: GlowUp Skin Serum
- Target audience: Women aged 25–40 interested in skincare
- Discount: 25% off for first-time buyers
- Launch date: July 10, 2025

Please write a marketing email using this information.
"""

In [28]:
def mail_chain():
    
    email_writer.receive(
    sender=user_input,
    message={"content": user_campaign_input}
    ),
    email_writer.register_reply(translator_writer,reply_func=translator_writer.generate_reply),
    translator_writer.register_reply(reviewer_writer,reply_func=reviewer_writer.generate_reply),
    reviewer_writer.register_reply(preview_sender,reply_func=preview_sender.generate_reply),
    preview_sender.register_reply(dispatcher,reply_func=dispatcher.generate_reply)

In [29]:
mail_chain()

user_input (to email_writer):


Here are the campaign details:
- Product name: GlowUp Skin Serum
- Target audience: Women aged 25–40 interested in skincare
- Discount: 25% off for first-time buyers
- Launch date: July 10, 2025

Please write a marketing email using this information.


--------------------------------------------------------------------------------
